In [108]:
import warnings
import argparse
import os.path as osp
import numpy as np


import tensorflow as tf
import numpy as np

from tf_lab.nips.helper import pclouds_centered_and_half_sphere
from general_tools.simpletons import iterate_in_chunks

from tf_lab.external.Chamfer_EMD_losses.tf_nndistance import nn_distance
from tf_lab.external.Chamfer_EMD_losses.tf_approxmatch import approx_match, match_cost

In [109]:
ref_file = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/03001627_train.npz'

gt_data = np.load(ref_file)  # Load Ground-Truth Data.
gt_data = gt_data[gt_data.keys()[0]]
gt_data = pclouds_centered_and_half_sphere(gt_data)

In [111]:
sample_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/r_gan/chair_mlp_disc_4_fc_gen_raw_gan_2048_pts'

In [112]:
epoch = 500
sample_file = osp.join(sample_dir, 'epoch_%d.npz' % (epoch,) )
sample_data = np.load(sample_file)
sample_data = sample_data[sample_data.keys()[0]]
sample_data = pclouds_centered_and_half_sphere(sample_data)    

In [116]:
def minimum_mathing_distance(sample_pcs, ref_pcs, batch_size, num_points, normalize=False, sess=None):
    ''' normalize (boolean): if True the Chamfer distance between two point-clouds is the average of matched 
                             point-distances. Alternatively, is their sum.
    '''
    
    if normalize:
        reducer = tf.reduce_mean
    else:
        reducer = tf.reduce_sum
    
    if sess is None:
        sess = tf.Session()
    
    v_ref = tf.placeholder(tf.float32, shape=(batch_size, num_points, 3))
    v_sample = tf.placeholder(tf.float32, shape=(batch_size, num_points, 3))
    
    n_ref = ref_pcs.shape[0]    
    matched_dists = np.zeros(n_ref)
    
    ref_to_s, _, s_to_ref, _ = nn_distance(v_ref, v_sample)                    
    chamfer_dist_batch = reducer(ref_to_s, 1) + reducer(s_to_ref, 1)            
    best_in_batch = tf.reduce_min(chamfer_dist_batch)
    
    n_total = len(ref_pcs)
    matched_dists = np.zeros(n_total)
    for i in xrange(n_total):
        print i
        best_in_all_batches = []
#         ch_dist_i = np.Inf
        for sample_chunk in iterate_in_chunks(sample_pcs, batch_size):
            ref_repeat = np.tile(ref_pcs[i], (batch_size, 1)).reshape(batch_size, num_points, 3)
            feed_dict = {v_ref: ref_repeat, v_sample: sample_chunk}            
            b = sess.run(best_in_batch, feed_dict=feed_dict)
            best_in_all_batches.append(b)
                        
        matched_dists[i] = np.min(best_in_all_batches)
        
    mmd = np.mean(matched_dists)
    return mmd, matched_dists

In [117]:
minimum_mathing_distance(sample_data[:1000], gt_data[:1000], 1000, 2048, normalize=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

(0.0022141417946550065,
 array([ 0.00171337,  0.00223664,  0.00271166,  0.00254655,  0.00149094,
         0.00220165,  0.0041354 ,  0.00079383,  0.00191371,  0.00260232,
         0.00167036,  0.00148901,  0.00087546,  0.00083565,  0.00273021,
         0.0010396 ,  0.00174953,  0.00288057,  0.00259997,  0.00497013,
         0.00366488,  0.00289693,  0.00125651,  0.01070827,  0.0016181 ,
         0.00090702,  0.00207431,  0.00370217,  0.00219172,  0.00337244,
         0.00249934,  0.00123288,  0.00114651,  0.00360795,  0.00167124,
         0.0011731 ,  0.00454075,  0.00226004,  0.00084558,  0.00229683,
         0.00285092,  0.00147335,  0.00077332,  0.0009403 ,  0.0012056 ,
         0.00097868,  0.00101022,  0.00156144,  0.00128786,  0.0018016 ,
         0.00201865,  0.0034445 ,  0.00141384,  0.00170906,  0.00660223,
         0.00255783,  0.00227203,  0.00221954,  0.00260906,  0.00156849,
         0.00200653,  0.00212466,  0.00303464,  0.0011255 ,  0.00541875,
         0.00217988,  0.001